# Лабораторная работа №3

ФИО: Поляховская Ольга Александровна  
Группа: БИВТ-22-СП-2

Исходные данные:
1. В [табличке](https://docs.google.com/spreadsheets/d/1k2MYXEp_OVQrink_XRC1PwoM0ZuNMeNz4VTWY7cI3hE/edit?usp=sharing) необходимо узнать название своего датасета 
2. Скачать нужны вам данные можно в [Google Drive](https://drive.google.com/drive/folders/1Phm-Fq1GL-VX7NS-DCEMRh_Fo1wU7DQ9?usp=sharing)
  
---
Теперь по пунктам, что я от вас жду:  
1. Загрузить необходимые данные к себе и считать (read) их в переменную.
2. Понять, у вас задача классификации (бинарной или многоклассовой) или регрессии (**если у вас многоклассовая классификация, прочтите P.S.S. внизу**).
3. Сделать предобработку данных:  
     1. Разделить выборку на тренировочную (train) и тестовую (test). _Обратите внимание, что обучать скейлеры и определять, какими значениями вы будете заполнять пропуски, вы будете на train выборке, а применять и на train, и на test_.
     2. Проверить пропуски в данных. Если они есть, заполнить одной из стратегий, предложенных в ноутбуке для семинара №3. P.S. Для численных и категориальных переменных будут разные стратегии.
     3. Отнормировать численные переменные (`StandardScaler`, `MinMaxScaler`).
     4. Закодировать категориальные признаки по одной из стратегий.
4. Обучить на тренировочном множестве:
     1. Линейную модель (`LogisticRegression`, `LinearRegression`)
     2. Деревянную модель (`DecisionTreeClassifier`, `DecisionTreeRegressor`) (тут советую попробовать разные глубины деревьев)
     3. K-ближайших соседей (`KNeighborsClassifier`, `KNeighborsRegressor`) (тут тоже есть смысл попробовать разные `k`)
     4. Случайный лес (`RandomForestClassifier`, `RandomForestRegressor`) 
5. Посчитайте метрики на train и test множествах:
     1. Для задачи классификации -- Accuracy, ROC-AUC (график + значение), PR-кривую (график), F1-score
     2. Для задачи регрессии -- MAE, RMSE, MAPE
6. Сравните метрики относительно train/test, так и относительно разных моделей. Ответьте на следующие вопросы:
     1. Какая модель справилась лучше с поставленной задачей?
     2. Имеет ли место переобучение?
     3. Имеет ли место недообучение?
     4. Как можно улучшить метрики моделей?


## 1. Пример импорта данных. Грузим данные

In [21]:
import pandas as pd
import math as m
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler

data = pd.read_csv('winequality-red.csv', sep=',')

data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [22]:
print(f'Уникальные значения целевой переменной: {data["quality"].nunique()}')
print(f'Размерность датасета: {data.shape}')

Уникальные значения целевой переменной: 6
Размерность датасета: (1599, 12)


## 2. Понимаем, какая перед нами задача

### Цель
Разработать модель машинного обучения для предсказания оценки красного вина на основе его химического состава и особенностях приготовления. Перед нами стоит задача регрессии
### Переменные
#### Целевая переменная
- `quality`: оценка вина, которую дали сомелье. Диапазон от 3 до 8

#### Признаки
- `fixed acidity`: Кислота 
- `volatile acidity`: Кислота
- `citric acid`: Содержание лимонной кислоты в вине
- `residual sugar`: Значение содержания остаточного сахара в вине
- `chlorides`: Процент содержания хлоридов
- `free sulfur dioxide`: 
- `density`: Плотность вина
- `pH`: Значение pH кислотности вина
- `sulphates`: Процент содержания сульфатов в вине
- `alcohol`: Процент содержания алкоголя в вине

## 3. Делаем предобработку данных

Проверить пропуски в данных

In [23]:
print(f'Пропуски в колонках датасета:\n{data.isnull().sum()}')

Пропуски в колонках датасета:
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64


Нормирование численных переменных

In [24]:
for elem in data.keys():
	data[elem] = MinMaxScaler().fit_transform(data[elem].values.reshape(-1, 1))
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846,0.4
1,0.283186,0.520548,0.00,0.116438,0.143573,0.338028,0.215548,0.494126,0.362205,0.209581,0.215385,0.4
2,0.283186,0.438356,0.04,0.095890,0.133556,0.197183,0.169611,0.508811,0.409449,0.191617,0.215385,0.4
3,0.584071,0.109589,0.56,0.068493,0.105175,0.225352,0.190813,0.582232,0.330709,0.149701,0.215385,0.6
4,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,0.141593,0.328767,0.08,0.075342,0.130217,0.436620,0.134276,0.354626,0.559055,0.149701,0.323077,0.4
1595,0.115044,0.294521,0.10,0.089041,0.083472,0.535211,0.159011,0.370778,0.614173,0.257485,0.430769,0.6
1596,0.150442,0.267123,0.13,0.095890,0.106845,0.394366,0.120141,0.416300,0.535433,0.251497,0.400000,0.6
1597,0.115044,0.359589,0.12,0.075342,0.105175,0.436620,0.134276,0.396476,0.653543,0.227545,0.276923,0.4


Разделение выборки на тренировочную и тестовую

In [25]:
from sklearn.model_selection import train_test_split

x = data[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']].values

y = data['quality'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

print(f'Тренировочная выборка x:\n{x_train}\n')
print(f'Тренировочная выборка y:\n {y_train}\n')
print(f'Тестовая выборка x:\n {x_test}\n')
print(f'Тестовая выборка y:\n {y_test}\n')
print(x_train.shape, x_test.shape)

Тренировочная выборка x:
[[0.31858407 0.41780822 0.21       ... 0.36220472 0.11377246 0.16923077]
 [0.26548673 1.         0.         ... 0.5984252  0.04191617 0.38461538]
 [0.65486726 0.34931507 0.5        ... 0.25984252 0.16167665 0.30769231]
 ...
 [0.50442478 0.15068493 0.52       ... 0.34645669 0.18562874 0.15384615]
 [0.12389381 0.20547945 0.19       ... 0.51181102 0.26946108 0.24615385]
 [0.46902655 0.42465753 0.28       ... 0.42519685 0.10778443 0.21538462]]

Тренировочная выборка y:
 [0.4 0.  0.2 ... 0.4 0.6 0.4]

Тестовая выборка x:
 [[0.15044248 0.26712329 0.13       ... 0.53543307 0.25149701 0.4       ]
 [0.23893805 0.2739726  0.32       ... 0.47244094 0.29341317 0.69230769]
 [0.38053097 0.11643836 0.35       ... 0.34645669 0.61676647 0.29230769]
 ...
 [0.15044248 0.26712329 0.13       ... 0.53543307 0.25149701 0.4       ]
 [0.37168142 0.32876712 0.29       ... 0.48818898 0.09580838 0.10769231]
 [0.61946903 0.24315068 0.4        ... 0.25984252 0.19161677 0.25128205]]

Тестова

## 4. Обучаем на тренировочном множестве

Линейная модель

In [26]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(x_train, y_train)

lr_pred_test = lr.predict(x_test)
lr_pred_train = lr.predict(x_train)

Деревянная модель

In [27]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth = 10)
tree.fit(x_train, y_train)

tree_pred_test = tree.predict(x_test)
tree_pred_train = tree.predict(x_train)

К ближайших соседей

In [28]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()
knn.fit(x_train, y_train)

knn_pred_test = knn.predict(x_test)
knn_pred_train = knn.predict(x_train)

Случайный лес

In [29]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()
rfr.fit(x_train, y_train)

rfr_pred_test = rfr.predict(x_test)
rfr_pred_train = rfr.predict(x_train)

## 5. Считаем метрики на train и test множесвах(MAE, rmse, MAPE)

In [30]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def errors(y, pred):
	mae = mean_absolute_error(y, pred)
	rmse = mean_squared_error(y, pred)**0.5
	y, pred = np.array(y), np.array(pred)
	non_zero_indices = y != 0
	mape = np.mean(np.abs((y[non_zero_indices] - pred[non_zero_indices]) / y[non_zero_indices]))*100
	print(f'MAE = {mae:.4f}; RMSE = {rmse:.4f}; MAPE = {mape:.4f}')
	return

print(f'\nМетрики линейной модели:\n На тестовой выборке:')
errors(y_test, lr_pred_test)
print(f'На тренировочной выборке:')
errors(y_train, lr_pred_train)

print(f'\nМетрики деревянной модели:\n На тестовой выборке:')
errors(y_test, tree_pred_test)
print(f'На тренировочной выборке:')
errors(y_train, tree_pred_train)

print(f'\nМетрики К ближайших соседей модели:\n На тестовой выборке:')
errors(y_test, knn_pred_test)
print(f'На тренировочной выборке:')
errors(y_train, knn_pred_train)

print(f'\nМетрики модели случайный лес:\n На тестовой выборке:')
errors(y_test, rfr_pred_test)
print(f'На тренировочной выборке:')
errors(y_train, rfr_pred_train)


Метрики линейной модели:
 На тестовой выборке:
MAE = 0.0964; RMSE = 0.1254; MAPE = 20.2160
На тренировочной выборке:
MAE = 0.1037; RMSE = 0.1321; MAPE = 21.8710

Метрики деревянной модели:
 На тестовой выборке:
MAE = 0.1043; RMSE = 0.1583; MAPE = 21.5397
На тренировочной выборке:
MAE = 0.0368; RMSE = 0.0673; MAPE = 7.9615

Метрики К ближайших соседей модели:
 На тестовой выборке:
MAE = 0.1015; RMSE = 0.1341; MAPE = 21.2621
На тренировочной выборке:
MAE = 0.0854; RMSE = 0.1156; MAPE = 18.2299

Метрики модели случайный лес:
 На тестовой выборке:
MAE = 0.0823; RMSE = 0.1137; MAPE = 17.1717
На тренировочной выборке:
MAE = 0.0330; RMSE = 0.0457; MAPE = 6.9604


## 6. Сравниваем метрики относительно train/test, так и относительно разных моделей.

1. Какая модель справилась лучше с поставленной задачей?

	Меньше всего различий в метриках у линейной модели, поэтому она справилась лучше других
2. Имеет ли место переобучение?

	Большинство моделей имеют более низкие метрики на тренировочной выборке, чем на тестовой, т.е. модели переобучены.
3. Имеет ли место недообучение?

	Метрики многих моделей не сильно отличаются отосительно тренировочной и тестовой выборках и значения в целом не высокие, поэтому недообучения быть не может
4. Как можно улучшить метрики моделей?

	Использовать больше данных на обучении
	Провести более тщательную предобработку параметров